# Exploitation Zone — Image Processing

This notebook handles the **image processing** step for the Exploitation Zone of our data pipeline.  
Its primary goal is to:

1. **Load images** from the Trusted Zone
2. **Generate embeddings** from the images in the Trusted Zone
3. **Store the embeddings** in ChromaDB



## 1. Setup and Configuration


In [ ]:
import os, io, json, re
from pathlib import PurePosixPath
from datetime import datetime, timezone
from typing import Dict, List, Set, Iterable

import boto3
from botocore.config import Config
from botocore.exceptions import ClientError
from dotenv import load_dotenv

from chromadb import PersistentClient
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction

import numpy as np
from PIL import Image

load_dotenv()

# S3 / MinIO Configuration
MINIO_USER     = os.getenv("MINIO_USER")
MINIO_PASSWORD = os.getenv("MINIO_PASSWORD")
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")

session = boto3.session.Session(
    aws_access_key_id=MINIO_USER,
    aws_secret_access_key=MINIO_PASSWORD,
    region_name="us-east-1"
)
s3 = session.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    config=Config(signature_version="s3v4", s3={"addressing_style": "path"})
)

# Paths and Buckets
TRUST_BUCKET        = "trusted-zone"
TRUST_IMAGES_PREFIX = "images"

# ChromaDB Persistent Client
PERSIST_DIR = os.getenv("CHROMA_PERSIST_DIR")

def utc_ts():
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H-%M-%SZ")

## 2. S3 Helper Functions

These utility functions provide a clean interface for S3 operations, handling common patterns like listing objects, checking existence, and copying files between buckets.


In [39]:
def list_all_jpg_keys(bucket: str, prefix: str):
    keys, token = [], None
    while True:
        kwargs = {"Bucket": bucket, "Prefix": prefix, "MaxKeys": 1000}
        if token: kwargs["ContinuationToken"] = token
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp.get("Contents", []):
            k = obj["Key"]
            if k.lower().endswith(".jpg") or k.lower().endswith(".jpeg"):
                # store keys relative to the prefix (cleaner IDs)
                keys.append(k)
                print(f"Found image: {keys[-1]}")
        token = resp.get("NextContinuationToken")
        if not token:
            break
    return keys

def get_np_from_minio(key: str) -> np.ndarray:
    obj = s3.get_object(Bucket=TRUST_BUCKET, Key=key)
    return np.array(Image.open(io.BytesIO(obj["Body"].read())).convert("RGB"))


## 3. Create and/or connect to the chromaDB


In [ ]:
chroma_client = PersistentClient(path=PERSIST_DIR)
ef_img = OpenCLIPEmbeddingFunction()

images_name = "trusted_zone_images"
multi_name = "trusted_zone_multimodal"

images_col = chroma_client.get_or_create_collection(
    name=images_name,
    embedding_function=ef_img,
    metadata={"modality": "image", "model": "OpenCLIP", "source": "minio"}
)
multi_col = chroma_client.get_or_create_collection(
    name=multi_name,
    embedding_function=ef_img,
    metadata={"modality": "multimodal", "model": "OpenCLIP", "source": "minio"}
)
print("✅ Chroma DB directory:", PERSIST_DIR)


✅ Chroma DB directory: ./chroma_exploitation


## 4. Generate embeddings and store in chromaDB


In [ ]:
image_keys = list_all_jpg_keys(TRUST_BUCKET, TRUST_IMAGES_PREFIX)

BATCH = 128
buf_ids, buf_imgs, buf_meta = [], [], []

for i, key in enumerate(image_keys, 1):
    try:
        arr = get_np_from_minio(key)
    except Exception:
        arr = np.full((224,224,3), 255, dtype=np.uint8)  # fallback

    buf_ids.append(key.replace("/", "__"))
    buf_imgs.append(arr)
    buf_meta.append({"type": "image", "object_key": key, "bucket": TRUST_BUCKET})

    if len(buf_ids) == BATCH or i == len(image_keys):
        images_col.add(ids=buf_ids, images=buf_imgs, metadatas=buf_meta)
        multi_col.add(ids=buf_ids, images=buf_imgs, metadatas=buf_meta)
        buf_ids, buf_imgs, buf_meta = [], [], []

print("✅ Embedded + stored all images in local Chroma")


Found image: images/image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$023bdcc87e1cb309921cc24051bd24e0__000b71bd25_0.jpg
Found image: images/image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$1131df21b2f8a0beef920711e0b8bde5__000bd531d5_0.jpg
Found image: images/image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$119b8b2249d9012935ba228549bba6c5__000bba053c_0.jpg
Found image: images/image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$142496c88baebda2c924c53461d0f78e__00073a6b36_0.jpg
Found image: images/image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$178c1af104b971fe99d70bfdaf0b9f10__00059b093b_0.jpg
Found image: images/image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$19214085d36fb535c7dbf24d178dea1b__000095fc1d_0.jpg
Found image: images/image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$1a426726a1e467b8e6360e980b03f611__00003a70b1_1.jpg
Found image: images/image$adsdb-multimodal-food-

## 5. Test retreaval


In [42]:
# Text → Image (cross-modal CLIP search)
res = images_col.query(
    query_texts=["pasta with tomato and basil"],
    n_results=5,
    include=["metadatas","distances"]
)
res


{'ids': [['images__image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$c286b2d709037f6206e7d3a8d89d4558__000dc3e7e7_0.jpg',
   'images__image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$ec753e257500b2d2c6455e8080e5f279__000c9dace9_0.jpg',
   'images__image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$f81a9c593da49674756ac22f1f61e0a2__000bcc1e31_0.jpg',
   'images__image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$26f424f36357fc1fbf130a1d6b56e90b__000b71bd25_1.jpg',
   'images__image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$46ffc3890fb1c38ca6bf1520471269cf__0003d5b120_0.jpg']],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'included': ['metadatas', 'distances'],
 'data': None,
 'metadatas': [[{'bucket': 'trusted-zone',
    'object_key': 'images/image$adsdb-multimodal-food-data-management$2025-10-25T17-44-21Z$c286b2d709037f6206e7d3a8d89d4558__000dc3e7e7_0.jpg'},
   {'object_key': 'images/image$adsdb-mul